In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
print(os.cpu_count())

import copy
import wandb
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
from torch import nn
from torchvision import models
import torch.backends.cudnn as cudnn

import albumentations as A
from albumentations.pytorch import ToTensorV2

from torcheval.metrics.functional import binary_auroc

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedGroupKFold

import albumentations as A
from albumentations.pytorch import ToTensorV2

from colorama import Fore, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

30


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.12 (you have 1.4.11). Upgrade using: pip install --upgrade albumentations


Using cuda device


In [3]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.set_float32_matmul_precision('highest')

# Set the random seed
set_seed(42)

In [4]:
def add_path(row):
    return f"../data/train-image/image/{row.isic_id}.jpg"

train_metadata_df = pd.read_csv("../data/stratified_5_fold_train_metadata.csv")
train_metadata_df["path"] = train_metadata_df.apply(lambda row: add_path(row), axis=1)

/tmp/ipykernel_1677165/1003576336.py:4: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_metadata_df = pd.read_csv("../data/stratified_5_fold_train_metadata.csv")


In [5]:
class SkinDataset(Dataset):
    def __init__(self, df: pd.DataFrame, transform=None, target_transform=None):
        assert "path" in df.columns
        assert "target" in df.columns

        self.paths = df.path.tolist()
        self.labels = df.target.tolist() # binary
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx: int):
        image = read_image(self.paths[idx]).to(torch.float32) / 255.0
        label = self.labels[idx] / 1.0
        if self.transform:
            image = image.numpy().transpose((1,2,0))
            image = self.transform(image=image)["image"]
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

    def get_class_samples(self, class_label):
        indices = [i for i, label in enumerate(self.labels) if label == class_label]
        return indices

In [6]:
transforms_train = A.Compose([
    A.Resize(124, 124),
    A.Normalize(
        # mean=(0.6962, 0.5209, 0.4193),
        # std=(0.1395, 0.1320, 0.1240)
    ),
    ToTensorV2(),
])

transforms_valid = A.Compose([
    A.Resize(124, 124),
    A.Normalize(
        # mean=(0.6962, 0.5209, 0.4193),
        # std=(0.1395, 0.1320, 0.1240)
    ),
    ToTensorV2(),
])

/home/ubuntu/.local/lib/python3.10/site-packages/pydantic/main.py:308: UserWarning: Pydantic serializer warnings:
  Expected `Union[float, json-or-python[json=list[float], python=list[float]]]` but got `tuple` - serialized value may not be as expected
  Expected `Union[float, json-or-python[json=list[float], python=list[float]]]` but got `tuple` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [7]:
def train_model(model, dataloader, criterion, optimizer, train_step, scheduler=None):
    model.train()  # Set model to training mode

    running_loss = 0.0
    running_auroc = 0.0

    for idx, (inputs, labels) in enumerate(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device).flatten().to(torch.float32)

        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            outputs = model(inputs).flatten()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        loss_val = loss.detach()
        running_loss += loss_val * inputs.size(0)

        auroc = binary_auroc(input=outputs, target=labels).item()
        running_auroc += auroc * inputs.size(0)

        if (idx + 1) % 10 == 0:
            train_step += 1
            wandb.log({"train_loss": loss_val, "train_auroc": auroc, "train_step": train_step})
            print(f"Train Batch Loss: {loss_val} | Train AUROC: {auroc}")

    epoch_loss = running_loss / dataset_sizes["train"]
    epoch_auroc = running_auroc / dataset_sizes["train"]

    return model, epoch_loss, epoch_auroc


def validate_model(model, dataloader, criterions, optimizer, valid_step):
    model.eval()

    running_loss = 0.0
    running_auroc = 0.0

    for idx, (inputs, labels) in enumerate(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device).to(torch.float32)

        optimizer.zero_grad()

        with torch.no_grad():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        
        loss_val = loss.detach()
        running_loss += loss_val * inputs.size(0)

        auroc = binary_auroc(input=outputs, target=labels).item()
        running_auroc += auroc * inputs.size(0)

        if (idx + 1) % 10 == 0:
            valid_step += 1
            wandb.log({"valid_loss": loss_val, "valid_auroc": auroc, "valid_step": valid_step})
            print(f"valid Batch Loss: {loss_val} | Valid AUROC: {auroc}")

    valid_loss = running_loss / dataset_sizes["val"]
    valid_auroc = running_auroc / dataset_sizes["val"]

    return model, valid_loss, valid_auroc

In [38]:
class SkinClassifier(nn.Module):
    def __init__(self, model_name='resnet18', freeze_backbone=False):
        super(SkinClassifier, self).__init__()
        
        # Load the specified pre-trained model
        if model_name == 'resnet18':
            self.backbone = models.resnet18(weights="IMAGENET1K_V1")
            if freeze_backbone:
                self.freeze_backbone()
            num_ftrs = self.backbone.fc.in_features
            self.backbone.fc = self.get_clf_head(num_ftrs, 1)
        elif model_name == 'convnext_tiny':
            self.backbone = models.convnext_tiny(weights="IMAGENET1K_V1")
            if freeze_backbone:
                self.freeze_backbone()
            num_ftrs = self.backbone.classifier[2].in_features
            self.backbone.classifier[2] = self.get_clf_head(num_ftrs, 1)
        elif model_name == "efficientnet_v2_s":
            self.backbone = models.efficientnet_v2_s(weights="IMAGENET1K_V1")
            if freeze_backbone:
                self.freeze_backbone()
            num_ftrs = self.backbone.classifier[1].in_features
            self.backbone.classifier[1] = self.get_clf_head(num_ftrs, 1)
        elif model_name == "mobilenet_v3_small":
            self.backbone = models.mobilenet_v3_small(weights="IMAGENET1K_V1")
            if freeze_backbone:
                self.freeze_backbone()
            num_ftrs = self.backbone.classifier[3].in_features
            self.backbone.classifier[3] = self.get_clf_head(num_ftrs, 1)
        else:
            raise ValueError(f"Model {model_name} not supported")        

    def forward(self, x):
        return self.backbone(x)
    
    def freeze_backbone(self):
        for param in self.backbone.parameters():
            param.requires_grad = False

    def get_clf_head(self, in_features, out_features, bias=None):
        return nn.Linear(in_features, out_features)

    def count_parameters(self):
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        non_trainable_params = sum(p.numel() for p in self.parameters() if not p.requires_grad)
        return trainable_params, non_trainable_params

In [ ]:
# Create the model
model = SkinClassifier(model_name='resnet18', freeze_backbone=True)
model = model.to(device)
model = torch.compile(model)
print(model)

# Example to get the number of trainable and non-trainable parameters
trainable_params, non_trainable_params = model.count_parameters()
print(f"Trainable parameters: {trainable_params}")
print(f"Non-trainable parameters: {non_trainable_params}")

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=0.001,
    weight_decay=1e-6
)

## Overfit on few samples :check:

In [ ]:
# train_df = train_metadata_df.loc[train_metadata_df.fold == 0] # using a subset for training
# valid_df = train_metadata_df.loc[train_metadata_df.fold == 1] # using another fold for validation

# num_workers = 24 # based on profiling

# train_dataset = SkinDataset(train_df.iloc[:10], transform=transforms_train)
# train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True, num_workers=num_workers, pin_memory=True, persistent_workers=True, drop_last=True)

# dataset_sizes = {"train": len(train_dataset)}
# dataset_sizes

In [ ]:
# run = wandb.init(project="isic_lesions_24", group="exps")
# wandb.define_metric("train_step")
# wandb.define_metric("valid_step")

# train_step = 0
# valid_step = 0

# best_epoch_auroc = -np.inf
# best_valid_loss = np.inf
# early_stopping_patience = 4
# epochs_no_improve = 0

# for epoch in range(100):
#     model_ft, epoch_loss, epoch_auroc = train_model(
#         model, train_dataloader, criterion, optimizer, train_step,
#     )

#     print(
#         f"Epoch: {epoch} | Train Loss: {epoch_loss} | AUROC: {epoch_auroc}"
#     )
#     wandb.log(
#         {
#             "epoch": epoch,
#             "epoch_loss": epoch_loss,
#             "epoch_auroc": epoch_auroc,
#         }
#     )

## Loss at init :check:

In [ ]:
-np.log(1/2)

## Train fully and see the results

In [ ]:
train_df = train_metadata_df.loc[train_metadata_df.fold == 0] # using a subset for training
valid_df = train_metadata_df.loc[train_metadata_df.fold == 1] # using another fold for validation

num_workers = 24 # based on profiling

train_dataset = SkinDataset(train_df, transform=transforms_train)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=num_workers, pin_memory=True, persistent_workers=True, drop_last=True)

valid_dataset = SkinDataset(valid_df, transform=transforms_valid)
valid_dataloader = DataLoader(valid_dataset, batch_size=128, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=True)

dataset_sizes = {"train": len(train_dataset), "val": len(valid_dataset)}
dataset_sizes

In [ ]:
def train_model(model, dataloader, criterion, optimizer, train_step, scheduler=None):
    model.train()  # Set model to training mode

    running_loss = 0.0
    running_auroc = 0.0

    for idx, (inputs, labels) in enumerate(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device).flatten().to(torch.float32)

        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            outputs = model(inputs).flatten()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        loss_val = loss.detach()
        running_loss += loss_val * inputs.size(0)

        auroc = binary_auroc(input=outputs, target=labels).item()
        running_auroc += auroc * inputs.size(0)

        if (idx + 1) % 10 == 0:
            train_step += 1
            wandb.log({"train_loss": loss_val, "train_auroc": auroc, "train_step": train_step})
            print(f"Train Batch Loss: {loss_val} | Train AUROC: {auroc}")

    epoch_loss = running_loss / dataset_sizes["train"]
    epoch_auroc = running_auroc / dataset_sizes["train"]

    return model, epoch_loss, epoch_auroc


def validate_model(model, dataloader, criterions, optimizer, valid_step):
    model.eval()

    running_loss = 0.0
    running_auroc = 0.0

    for idx, (inputs, labels) in enumerate(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device).flatten().to(torch.float32)

        optimizer.zero_grad()

        with torch.no_grad():
            outputs = model(inputs).flatten()
            loss = criterion(outputs, labels)
        
        loss_val = loss.detach()
        running_loss += loss_val * inputs.size(0)

        auroc = binary_auroc(input=outputs, target=labels).item()
        running_auroc += auroc * inputs.size(0)

        if (idx + 1) % 10 == 0:
            valid_step += 1
            wandb.log({"valid_loss": loss_val, "valid_auroc": auroc, "valid_step": valid_step})
            print(f"valid Batch Loss: {loss_val} | Valid AUROC: {auroc}")

    valid_loss = running_loss / dataset_sizes["val"]
    valid_auroc = running_auroc / dataset_sizes["val"]

    return model, valid_loss, valid_auroc

In [ ]:
run = wandb.init(project="isic_lesions_24", job_type="pretrain")

model_name = "resnet18"
exp_name = "train_linear"

wandb.define_metric("train_step")
wandb.define_metric("valid_step")

train_step = 0
valid_step = 0

best_epoch_auroc = -np.inf
best_valid_loss = np.inf
early_stopping_patience = 4
epochs_no_improve = 0

for epoch in range(40):
    model_ft, epoch_loss, epoch_train_auroc = train_model(
        model, train_dataloader, criterion, optimizer, train_step,
    )

    model_ft, valid_loss, epoch_valid_auroc = validate_model(
        model, valid_dataloader, criterion, optimizer, valid_step
    )

    wandb.log(
        {
            "epoch": epoch,
            "epoch_loss": epoch_loss,
            "epoch_val_loss": valid_loss,
            "epoch_train_auroc": epoch_train_auroc,
            "epoch_valid_auroc": epoch_valid_auroc
        }
    )

    print(
        f"Epoch: {epoch} | Train Loss: {epoch_loss} | Valid Loss: {valid_loss}\n"
    )
    print(
        f"Epoch: {epoch} | Train AUROC: {epoch_train_auroc} | Valid AUROC: {epoch_valid_auroc}\n"
    )

    # earlystopping dependent on validation loss
    if best_valid_loss >= valid_loss:
        print(f"{b_}Validation Loss Improved ({best_valid_loss} ---> {valid_loss}){sr_}")
        
        # checkpointing
        best_model_wts = copy.deepcopy(model_ft.state_dict())
        PATH = f"../models/{model_name}_{exp_name}_valid_loss{valid_loss}_epoch{epoch}.bin"
        torch.save(model_ft.state_dict(), PATH)
        # Save a model file from the current directory
        print(f"{b_}Model Saved{sr_}")
        best_valid_loss = valid_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= early_stopping_patience:
        print(
            f"{b_}Early stopping triggered after {epochs_no_improve} epochs with no improvement.{sr_}"
        )
        break

## Switch various models and train with frozen backbone - find best model :check:

## See the effect of batch size - try to consume the most GPU allocation

## Experiment with bias of the linear layer

In [39]:
model = SkinClassifier(model_name='efficientnet_v2_s', freeze_backbone=True)
model

SkinClassifier(
  (backbone): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): FusedMBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        )
        (1): FusedMBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): BatchNor

In [10]:
train_df = train_metadata_df.loc[train_metadata_df.fold == 0] # using a subset for training
valid_df = train_metadata_df.loc[train_metadata_df.fold == 1] # using another fold for validation

num_workers = 24 # based on profiling

train_dataset = SkinDataset(train_df, transform=transforms_train)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=num_workers, pin_memory=True, persistent_workers=True, drop_last=True)

In [12]:
imgs, labels = next(iter(train_dataloader))
imgs.shape, labels.shape

(torch.Size([128, 3, 124, 124]), torch.Size([128]))

In [40]:
# one sample prediction (init)

torch.sigmoid(model(imgs))

tensor([[0.5396],
        [0.5418],
        [0.5494],
        [0.5319],
        [0.5616],
        [0.4956],
        [0.4204],
        [0.5608],
        [0.4934],
        [0.4083],
        [0.3875],
        [0.4720],
        [0.4943],
        [0.5333],
        [0.5021],
        [0.5034],
        [0.4184],
        [0.5353],
        [0.4961],
        [0.5430],
        [0.5465],
        [0.4134],
        [0.5422],
        [0.4114],
        [0.5033],
        [0.4280],
        [0.4917],
        [0.5258],
        [0.6553],
        [0.5414],
        [0.4480],
        [0.5060],
        [0.5974],
        [0.6647],
        [0.5383],
        [0.5049],
        [0.4727],
        [0.4645],
        [0.4675],
        [0.4994],
        [0.5221],
        [0.4785],
        [0.5294],
        [0.4948],
        [0.4774],
        [0.4517],
        [0.4111],
        [0.5610],
        [0.4514],
        [0.5104],
        [0.5225],
        [0.6313],
        [0.4236],
        [0.4934],
        [0.4876],
        [0

In [41]:
neg_samples = len(train_dataloader.dataset.get_class_samples(0))
pos_samples = len(train_dataloader.dataset.get_class_samples(1))
print(neg_samples, pos_samples)

neg_wts = 1 / neg_samples
pos_wts = 1 / pos_samples
print(neg_wts, pos_wts)

print("ratio of pos/neg: ", pos_samples/neg_samples)

80134 77
1.2479097511667955e-05 0.012987012987012988
ratio of pos/neg:  0.0009608905083984327


In [42]:
import torch
import torch.nn as nn
import math

# Probability of a positive sample
p_positive = pos_samples / (neg_samples + pos_samples)

# Calculate the bias for the logit
bias_value = math.log(p_positive / (1 - p_positive))

print(f"Calculated bias value: {bias_value}")

Calculated bias value: -6.947650090554344


In [43]:
class SkinClassifier(nn.Module):
    def __init__(self, model_name='resnet18', freeze_backbone=False, bias_value=None):
        super(SkinClassifier, self).__init__()
        
        # Load the specified pre-trained model
        if model_name == 'resnet18':
            self.backbone = models.resnet18(weights="IMAGENET1K_V1")
            if freeze_backbone:
                self.freeze_backbone()
            num_ftrs = self.backbone.fc.in_features
            self.backbone.fc = self.get_clf_head(num_ftrs, 1, bias_value)
        elif model_name == 'convnext_tiny':
            self.backbone = models.convnext_tiny(weights="IMAGENET1K_V1")
            if freeze_backbone:
                self.freeze_backbone()
            num_ftrs = self.backbone.classifier[2].in_features
            self.backbone.classifier[2] = self.get_clf_head(num_ftrs, 1, bias_value)
        elif model_name == "efficientnet_v2_s":
            self.backbone = models.efficientnet_v2_s(weights="IMAGENET1K_V1")
            if freeze_backbone:
                self.freeze_backbone()
            num_ftrs = self.backbone.classifier[1].in_features
            self.backbone.classifier[1] = self.get_clf_head(num_ftrs, 1, bias_value)
        elif model_name == "mobilenet_v3_small":
            self.backbone = models.mobilenet_v3_small(weights="IMAGENET1K_V1")
            if freeze_backbone:
                self.freeze_backbone()
            num_ftrs = self.backbone.classifier[3].in_features
            self.backbone.classifier[3] = self.get_clf_head(num_ftrs, 1, bias_value)
        else:
            raise ValueError(f"Model {model_name} not supported")        

    def forward(self, x):
        return self.backbone(x)
    
    def freeze_backbone(self):
        for param in self.backbone.parameters():
            param.requires_grad = False

    def get_clf_head(self, in_features, out_features, bias_value=None):
        head = nn.Linear(in_features, out_features)
        nn.init.constant_(head.bias, bias_value)
        return head

    def count_parameters(self):
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        non_trainable_params = sum(p.numel() for p in self.parameters() if not p.requires_grad)
        return trainable_params, non_trainable_params

In [44]:
model = SkinClassifier(model_name='efficientnet_v2_s', freeze_backbone=True, bias_value=bias_value)
model

SkinClassifier(
  (backbone): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): FusedMBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        )
        (1): FusedMBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): BatchNor

In [45]:
torch.sigmoid(model(imgs))

tensor([[0.0009],
        [0.0008],
        [0.0015],
        [0.0010],
        [0.0008],
        [0.0016],
        [0.0011],
        [0.0009],
        [0.0009],
        [0.0011],
        [0.0013],
        [0.0014],
        [0.0008],
        [0.0010],
        [0.0010],
        [0.0015],
        [0.0023],
        [0.0011],
        [0.0009],
        [0.0010],
        [0.0015],
        [0.0009],
        [0.0008],
        [0.0013],
        [0.0008],
        [0.0013],
        [0.0008],
        [0.0008],
        [0.0014],
        [0.0010],
        [0.0009],
        [0.0011],
        [0.0013],
        [0.0013],
        [0.0019],
        [0.0008],
        [0.0015],
        [0.0012],
        [0.0009],
        [0.0012],
        [0.0007],
        [0.0007],
        [0.0018],
        [0.0008],
        [0.0011],
        [0.0015],
        [0.0014],
        [0.0012],
        [0.0008],
        [0.0018],
        [0.0016],
        [0.0011],
        [0.0010],
        [0.0007],
        [0.0007],
        [0